In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from transformers import ViTModel, ViTConfig
import os
from tqdm import tqdm
from PIL import Image
import torch.nn.functional as F

class UNetGenerator(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(UNetGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),  # 224x224 -> 112x112
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),             # 112x112 -> 56x56
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=2, padding=1),            # 56x56 -> 28x28
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=2, padding=1),            # 28x28 -> 14x14
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        self.middle = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=1, padding=1),  # Keeps size at 14x14
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=1, padding=1),  # Keeps size at 14x14
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 14x14 -> 28x28
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),   # 28x28 -> 56x56
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),   # 56x56 -> 112x112
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, output_channels, kernel_size=4, stride=2, padding=1),  # 112x112 -> 224x224
            nn.Tanh(),
        )

    def forward(self, x):
        # Encoder
        enc = self.encoder(x)
        #print(f"Shape after encoder: {enc.shape}")  # Check encoder output shape
        
        # Middle
        mid = self.middle(enc)
        #print(f"Shape after middle: {mid.shape}")  # Check shape after middle layer
        
        # Decoder
        dec = self.decoder(mid)
        #print(f"Shape after decoder: {dec.shape}")  # Final generator output shape
        return dec


class Discriminator(nn.Module):
    def __init__(self, input_channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 1, kernel_size=4, stride=2, padding=0),  # Ensures output is [B, 1, 1, 1]
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        return self.model(x)

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

class KITTIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = sorted([os.path.join(root_dir, img) for img in os.listdir(root_dir) if img.endswith('.png') or img.endswith('.jpg')])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0  # Dummy label as GANs don't require labels

dataset = KITTIDataset("/kaggle/input/kitti-dataset/data_object_image_2/training/image_2", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


# Initialize Models
generator = UNetGenerator(input_channels=128, output_channels=3).cuda()
discriminator = Discriminator(input_channels=3).cuda()
vit_config = ViTConfig(hidden_size=128, num_attention_heads=4, num_hidden_layers=6)
vit_encoder = ViTModel(vit_config).cuda()

# Optimizers
opt_gen = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_disc = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss Functions
adversarial_loss = nn.BCELoss()
reconstruction_loss = nn.MSELoss()

# Training Loop
epochs = 100
for epoch in range(epochs):
    for real_images, _ in tqdm(dataloader):
        real_images = real_images.cuda()
        batch_size = real_images.size(0)
        valid = torch.ones((batch_size, 1)).cuda()
        fake = torch.zeros((batch_size, 1)).cuda()

        # Train Generator
        opt_gen.zero_grad()

        #print(f"Input shape: {real_images.shape}")  # Shape of input image batch

        # Get the output from ViT encoder
        output = vit_encoder(real_images).last_hidden_state
        #print(f"Output shape (ViT): {output.shape}")  # ViT encoder output
        
        # Exclude the CLS token
        patch_embeddings = output[:, 1:, :]  # Remove the CLS token
        #print(f"Patch embeddings shape: {patch_embeddings.shape}")  # After excluding CLS token
        
        # Reshape the output to feed into the generator
        num_patches = int(patch_embeddings.shape[1] ** 0.5)  # Compute square root of number of patches
        z = patch_embeddings.permute(0, 2, 1).view(batch_size, 128, num_patches, num_patches)
        #print(f"Final tensor shape (z): {z.shape}")  # After reshaping for generator

        # After shaping z to [batch, 128, 14, 14], upsample it:
        z_upsampled = F.interpolate(z, size=(224, 224), mode='bilinear', align_corners=False)
        gen_images = generator(z_upsampled)

        #print(f"Generated images shape: {gen_images.shape}")


        valid = torch.ones_like(discriminator(gen_images)).cuda()  # Match shape dynamically
        fake = torch.zeros_like(discriminator(gen_images)).cuda()  # Match shape dynamically


        #print(f"Discriminator output shape: {discriminator(gen_images).shape}")
        #print(f"Valid tensor shape: {valid.shape}")

        
        g_loss = adversarial_loss(discriminator(gen_images), valid) + reconstruction_loss(gen_images, real_images)
        g_loss.backward()
        opt_gen.step()

        # Train Discriminator
        opt_disc.zero_grad()
        real_loss = adversarial_loss(discriminator(real_images), valid)
        fake_loss = adversarial_loss(discriminator(gen_images.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        opt_disc.step()

    print(f"Epoch {epoch}/{epochs} | Generator Loss: {g_loss.item()} | Discriminator Loss: {d_loss.item()}")

    # Save model checkpoint
    if epoch % 10 == 0:
        torch.save(generator.state_dict(), f"generator_epoch_{epoch}.pth")

100%|██████████| 234/234 [03:01<00:00,  1.29it/s]


Epoch 0/100 | Generator Loss: 1.0204671621322632 | Discriminator Loss: 0.5757521390914917


100%|██████████| 234/234 [02:04<00:00,  1.88it/s]


Epoch 1/100 | Generator Loss: 0.8944224119186401 | Discriminator Loss: 0.6046009659767151


100%|██████████| 234/234 [02:09<00:00,  1.81it/s]


...

Epoch 97/100 | Generator Loss: 0.8222793936729431 | Discriminator Loss: 0.685386061668396


100%|██████████| 234/234 [02:05<00:00,  1.87it/s]


Epoch 98/100 | Generator Loss: 0.7330008745193481 | Discriminator Loss: 0.6944208741188049


100%|██████████| 234/234 [02:03<00:00,  1.89it/s]


Epoch 99/100 | Generator Loss: 0.7911421656608582 | Discriminator Loss: 0.6760014295578003


### Load the model and test

In [31]:
import torch
from torchvision.utils import save_image

# Assume you are in the same environment where UNetGenerator is defined
# or have imported it from your module.

# Create a new instance of the generator and load the weights
generator = UNetGenerator(input_channels=128, output_channels=3).cuda()
checkpoint_path = "/kaggle/working/generator_epoch_90.pth"  
generator.load_state_dict(torch.load(checkpoint_path))
generator.eval()


/tmp/ipykernel_23/4223602956.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(checkpoint_path))


UNetGenerator(
  (encoder): Sequential(
    (0): Conv2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (middle): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(1

#### Using ViT. Requires sample image

In [33]:
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

# Load ViT and set to eval mode
vit_encoder = ViTModel(ViTConfig(hidden_size=128, num_attention_heads=4, num_hidden_layers=6)).cuda()
vit_encoder.eval()

# Same transformation used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load a sample image
test_image = Image.open("/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000000.png").convert("RGB")
test_image = transform(test_image).unsqueeze(0).cuda()  # shape: [1, 3, 224, 224]

# Get ViT embeddings
with torch.no_grad():
    output = vit_encoder(test_image).last_hidden_state  # shape: [1, 197, 128]

# Remove CLS token
patch_embeddings = output[:, 1:, :]  # shape: [1, 196, 128]

# Reshape into spatial dimensions
num_patches = int(patch_embeddings.shape[1] ** 0.5)  # Should be 14 for 14x14
z = patch_embeddings.permute(0, 2, 1).view(1, 128, num_patches, num_patches)  # [1,128,14,14]

# Upsample to 224x224 if that's what your generator expects
z_upsampled = F.interpolate(z, size=(224, 224), mode='bilinear', align_corners=False)  # [1,128,224,224]

# Generate image
with torch.no_grad():
    gen_images = generator(z_upsampled)  # [1, 3, 224, 224]

# Save the generated image
# If you normalized your data with mean=0.5 and std=0.5, convert back
gen_images = gen_images * 0.5 + 0.5
save_image(gen_images, "generated_sample.png")


#### Using noise

In [36]:
import torch
from torchvision.utils import save_image
import torch.nn.functional as F

# Create random input (matching the shape generator expects)
# The generator expects [N, 128, 224, 224] since we upsampled z
z_random = torch.randn(1, 128, 224, 224).cuda()

with torch.no_grad():
    gen_images = generator(z_random)  # [1, 3, 224, 224]

# Convert back to [0,1] range if you used the same normalization
gen_images = gen_images * 0.5 + 0.5

# Save the generated image
save_image(gen_images, "generated_sample_random.png")
